# Ejemplo con una sola capa

In [5]:
# ================================================
# Ejemplo básico de una capa densa con ReLU en Python
# ================================================

import numpy as np
import pandas as pd

# 1) Definir las entradas (x), los pesos (W) y los bias (b)
x = np.array([2, 1, 3])   # Entradas

W = np.array([            # Pesos de cada neurona (cada fila es una neurona)
    [1, -1, 1],
    [1,  1, 0],
    [0,  1, 1],
    [1,  0, 1],
])

b = np.array([-5, 0, 1, -2])  # Bias para cada neurona

# 2) Calcular pre-activación: z = W·x + b
z = W @ x + b  # @ es el operador de producto matricial en Python

# 3) Aplicar la función de activación ReLU
a = np.maximum(z, 0)  # ReLU: max(0, z)

# 4) Crear una tabla para entender cada paso
df = pd.DataFrame(W, columns=['w1', 'w2', 'w3'])
df['bias'] = b
df['w·x (sin bias)'] = (W @ x)  # Producto de pesos por entradas, sin bias
df['z = w·x + b'] = z
df['a = ReLU(z)'] = a

# Mostrar tabla en Colab
df

,w1,w2,w3,bias,w·x (sin bias),z = w·x + b,a = ReLU(z)
0,1,-1,1,-5,4,-1,0
1,1,1,0,0,3,3,3
2,0,1,1,1,4,5,5
3,1,0,1,-2,5,3,3


# Ejemplo refactorizado como Clase

In [6]:
# ================================================
# Clase DenseLayer (capa densa con activación ReLU)
# ================================================

import numpy as np
import pandas as pd

class DenseLayer:
    def __init__(self, weights, bias, activation="relu"):
        """
        Inicializa la capa densa.
        - weights: matriz (n_neuronas x n_entradas)
        - bias: vector (n_neuronas)
        - activation: función de activación ("relu" por defecto)
        """
        self.W = np.array(weights)
        self.b = np.array(bias)
        self.activation = activation.lower()

    def forward(self, x):
        """
        Calcula la salida de la capa:
        z = W·x + b
        a = activation(z)
        """
        self.x = np.array(x)  # guardamos la entrada
        self.z = self.W @ self.x + self.b  # pre-activación

        # Aplicamos la activación
        if self.activation == "relu":
            self.a = np.maximum(self.z, 0)
        elif self.activation == "sigmoid":
            self.a = 1 / (1 + np.exp(-self.z))
        elif self.activation == "tanh":
            self.a = np.tanh(self.z)
        elif self.activation == "linear":
            self.a = self.z
        else:
            raise ValueError(f"Activación {self.activation} no implementada")

        return self.a

    def summary(self):
        """
        Devuelve un DataFrame con el detalle de cálculos de la capa.
        """
        df = pd.DataFrame(self.W, columns=[f"w{i+1}" for i in range(self.W.shape[1])])
        df["bias"] = self.b
        df["w·x (sin bias)"] = self.W @ self.x
        df["z = w·x + b"] = self.z
        df[f"a = {self.activation}(z)"] = self.a
        return df

# ================================================
# Ejemplo con los datos
# ================================================
x = [2, 1, 3]   # entradas
W = [
    [1, -1, 1],
    [1,  1, 0],
    [0,  1, 1],
    [1,  0, 1],
]
b = [-5, 0, 1, -2]

# Crear la capa y hacer forward
layer1 = DenseLayer(W, b, activation="relu")
output = layer1.forward(x)

print("Salida de la capa:", output)
layer1.summary()

Salida de la capa: [0 3 5 3]


,w1,w2,w3,bias,w·x (sin bias),z = w·x + b,a = relu(z)
0,1,-1,1,-5,4,-1,0
1,1,1,0,0,3,3,3
2,0,1,1,1,4,5,5
3,1,0,1,-2,5,3,3


# Ejemplo multicapa con reutilización de la Clase

In [7]:
# ================================================
# Clase DenseLayer (igual que antes)
# ================================================

import numpy as np
import pandas as pd

class DenseLayer:
    def __init__(self, weights, bias, activation="relu"):
        self.W = np.array(weights)
        self.b = np.array(bias)
        self.activation = activation.lower()

    def forward(self, x):
        self.x = np.array(x)
        self.z = self.W @ self.x + self.b

        if self.activation == "relu":
            self.a = np.maximum(self.z, 0)
        elif self.activation == "sigmoid":
            self.a = 1 / (1 + np.exp(-self.z))
        elif self.activation == "tanh":
            self.a = np.tanh(self.z)
        elif self.activation == "linear":
            self.a = self.z
        else:
            raise ValueError(f"Activación {self.activation} no implementada")

        return self.a

    def summary(self):
        df = pd.DataFrame(self.W, columns=[f"w{i+1}" for i in range(self.W.shape[1])])
        df["bias"] = self.b
        df["w·x (sin bias)"] = self.W @ self.x
        df["z = w·x + b"] = self.z
        df[f"a = {self.activation}(z)"] = self.a
        return df

# ================================================
# Clase MLP (Perceptrón Multicapa)
# ================================================

class MLP:
    def __init__(self, layers_config):
        """
        layers_config: lista de diccionarios, cada uno con:
          - "weights": matriz de pesos
          - "bias": vector de bias
          - "activation": función de activación
        """
        self.layers = []
        for cfg in layers_config:
            layer = DenseLayer(cfg["weights"], cfg["bias"], cfg.get("activation", "relu"))
            self.layers.append(layer)

    def forward(self, x):
        """
        Pasa la entrada por todas las capas secuencialmente
        """
        for i, layer in enumerate(self.layers, start=1):
            x = layer.forward(x)
            print(f"➡️ Salida de la capa {i}: {x}")
        return x

    def summary(self):
        """
        Devuelve un resumen detallado de todas las capas
        """
        for i, layer in enumerate(self.layers, start=1):
            print(f"\n📌 Resumen de la capa {i}:")
            display(layer.summary())

# ================================================
# Ejemplo de uso con 2 capas ocultas + 1 capa de salida
# ================================================

# Entradas
x = [2, 1, 3]

# Configuración de la red
layers_config = [
    {
        "weights": [
            [1, -1, 1],
            [1,  1, 0],
            [0,  1, 1],
            [1,  0, 1],
        ],
        "bias": [-5, 0, 1, -2],
        "activation": "relu"
    },
    {
        "weights": [
            [1, 1, 1, 1],
            [0, 1, -1, 1],
        ],
        "bias": [0, 2],
        "activation": "relu"
    },
    {
        "weights": [
            [1, -1],
        ],
        "bias": [0],
        "activation": "linear"  # Capa de salida
    }
]

# Crear red y ejecutar forward
mlp = MLP(layers_config)
output = mlp.forward(x)

print("\n🎯 Salida final de la red:", output)


➡️ Salida de la capa 1: [0 3 5 3]
➡️ Salida de la capa 2: [11  3]
➡️ Salida de la capa 3: [8]

🎯 Salida final de la red: [8]


# Ejemplo multicapa con varias entradas

In [8]:
import numpy as np
import pandas as pd

# ================================================
# Función auxiliar para ReLU
def relu(x):
    return np.maximum(x, 0)

# ================================================
# Datos
X = np.array([[3, 4, 5],   # Ejemplo 1
              [5, 4, 3]])  # Ejemplo 2

W = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0],
    [1, 1, 0],
    [0, 1, 1],
])

b = np.zeros(W.shape[0])   # bias = 0

# ================================================
# Forward de UNA capa densa
Z = X @ W.T + b    # (2x3) · (3x5)ᵀ = (2x5)
A = relu(Z)

# Mostrar resultados paso a paso
df = pd.DataFrame(Z, columns=[f"neuron_{i+1}" for i in range(W.shape[0])])
df_relu = pd.DataFrame(A, columns=[f"neuron_{i+1}" for i in range(W.shape[0])])

print("🔹 Pre-activaciones (z = W·x + b):")
display(df)

print("🔹 Salida tras ReLU:")
display(df_relu)


🔹 Pre-activaciones (z = W·x + b):


,neuron_1,neuron_2,neuron_3,neuron_4,neuron_5
0,5.0,4.0,3.0,7.0,9.0
1,3.0,4.0,5.0,9.0,7.0


🔹 Salida tras ReLU:


,neuron_1,neuron_2,neuron_3,neuron_4,neuron_5
0,5.0,4.0,3.0,7.0,9.0
1,3.0,4.0,5.0,9.0,7.0


# Ejemplo multicapa con la Clase

In [11]:
import numpy as np
import pandas as pd

# Función ReLU
def relu(x):
    return np.maximum(x, 0)

# Datos (batch de 2 ejemplos)
X = np.array([[3, 4, 5],
              [5, 4, 3]])  # (2,3)

# Pesos dados por capas
W1 = np.array([[0, 0, 1],
               [0, 1, 0],
               [1, 0, 0],
               [1, 1, 0],
               [0, 1, 1]])    # (5,3)

W2 = np.array([[1, 1, -1, 0, 0],
               [0, 0,  1, 1, -1]])  # (2,5)

W3 = np.array([[1, 1],
               [1, -1],
               [1, 2]])  # (3,2)

W4 = np.array([[1, -1, 0],
               [0, -1, 1]])  # (2,3)

W5 = np.array([[0, 1],
               [1, 0]])  # (2,2)

W6 = np.array([[1, -1],
               [1, 1]])  # (2,2)

W7 = np.array([1, -1])  # (2,) -> treated as (1,2) for final linear comb.

# No hay bias (o bias = 0)
# Forward paso a paso
Z1 = X @ W1.T        # (2,5)
A1 = relu(Z1)

Z2 = A1 @ W2.T       # (2,2)
A2 = relu(Z2)

Z3 = A2 @ W3.T       # (2,3)
A3 = relu(Z3)

Z4 = A3 @ W4.T       # (2,2)
A4 = relu(Z4)

Z5 = A4 @ W5.T       # (2,2)
A5 = relu(Z5)

Z6 = A5 @ W6.T       # (2,2)
A6 = relu(Z6)

Z7 = A6 @ W7        # (2,) final linear output (no ReLU by default)
# If you prefer to keep it as (2,1) shape, do: Z7 = A6 @ W7.reshape(-1,1)

# Mostrar todo en forma legible con pandas
def df_arr(arr, name):
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    cols = [f"{name}_{i+1}" for i in range(arr.shape[1])]
    return pd.DataFrame(arr, columns=cols)

print("ENTRADAS (X):")
display(pd.DataFrame(X, columns=['x1','x2','x3']))

print("\n--- Capa 1 ---")
print("Z1 (pre-activación) = X @ W1.T")
display(df_arr(Z1, "z1"))
print("A1 = ReLU(Z1)")
display(df_arr(A1, "a1"))

print("\n--- Capa 2 ---")
print("Z2 = A1 @ W2.T")
display(df_arr(Z2, "z2"))
print("A2 = ReLU(Z2)")
display(df_arr(A2, "a2"))

print("\n--- Capa 3 ---")
print("Z3 = A2 @ W3.T")
display(df_arr(Z3, "z3"))
print("A3 = ReLU(Z3)")
display(df_arr(A3, "a3"))

print("\n--- Capa 4 ---")
print("Z4 = A3 @ W4.T")
display(df_arr(Z4, "z4"))
print("A4 = ReLU(Z4)")
display(df_arr(A4, "a4"))

print("\n--- Capa 5 ---")
print("Z5 = A4 @ W5.T")
display(df_arr(Z5, "z5"))
print("A5 = ReLU(Z5)")
display(df_arr(A5, "a5"))

print("\n--- Capa 6 ---")
print("Z6 = A5 @ W6.T")
display(df_arr(Z6, "z6"))
print("A6 = ReLU(Z6)")
display(df_arr(A6, "a6"))

print("\n--- Capa 7 (salida final lineal) ---")
print("Z7 = A6 @ W7  (vector final)")
display(pd.DataFrame(Z7, columns=['output']))

# También imprimo arrays simples al final
print("\nResumen (arrays):")
print("A1:", A1.tolist())
print("A2:", A2.tolist())
print("A3:", A3.tolist())
print("A4:", A4.tolist())
print("A5:", A5.tolist())
print("A6:", A6.tolist())
print("Z7 (final):", Z7.tolist())


ENTRADAS (X):


,x1,x2,x3
0,3,4,5
1,5,4,3



--- Capa 1 ---
Z1 (pre-activación) = X @ W1.T


,z1_1,z1_2,z1_3,z1_4,z1_5
0,5,4,3,7,9
1,3,4,5,9,7


A1 = ReLU(Z1)


...

A6 = ReLU(Z6)


,a6_1,a6_2
0,1,5
1,7,25



--- Capa 7 (salida final lineal) ---
Z7 = A6 @ W7  (vector final)


,output
0,-4
1,-18



Resumen (arrays):
A1: [[5, 4, 3, 7, 9], [3, 4, 5, 9, 7]]
A2: [[6, 1], [2, 7]]
A3: [[7, 5, 8], [9, 0, 16]]
A4: [[2, 3], [9, 16]]
A5: [[3, 2], [16, 9]]
A6: [[1, 5], [7, 25]]
Z7 (final): [-4, -18]
